<a href="https://colab.research.google.com/github/Sireesha024/CS59000_06_Ethical_Artificial_Intelligence/blob/main/ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install kagglehub

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("thovylnguyn/fakenewscorpus-full")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/fakenewscorpus-full


In [6]:
import pandas as pd
df = pd.read_csv("/kaggle/input/fakenewscorpus-full/news_cleaned_big.csv")

<ipython-input-6-9a1aa3755189>:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/kaggle/input/fakenewscorpus-full/news_cleaned_big.csv")


In [7]:
df.head()

,Unnamed: 0,id,domain,type,url,content,scraped_at,inserted_at,updated_at,title,authors,keywords,meta_keywords,meta_description,tags,summary,source
0,0,2,express.co.uk,rumor,https://www.express.co.uk/news/science/738402/...,"Life is an illusion, at least on a quantum lev...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Is life an ILLUSION? Researchers prove 'realit...,Sean Martin,NaN,[''],THE UNIVERSE ceases to exist when we are not l...,NaN,NaN,NaN
1,1,6,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,"Unfortunately, he hasn’t yet attacked her for ...",2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN
2,2,7,barenakedislam.com,hate,http://barenakedislam.com/category/donald-trum...,The Los Angeles Police Department has been den...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,Donald Trump,"Linda Rivera, Conrad Calvano, Az Gal, Lincoln ...",NaN,[''],NaN,NaN,NaN,NaN
3,3,8,barenakedislam.com,hate,http://barenakedislam.com/2017/12/24/more-winn...,The White House has decided to quietly withdra...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"MORE WINNING! Israeli intelligence source, DEB...","Cleavis Nowell, Cleavisnowell, Clarence J. Fei...",NaN,[''],NaN,NaN,NaN,NaN
4,4,9,barenakedislam.com,hate,http://barenakedislam.com/2017/12/25/oh-trump-...,“The time has come to cut off the tongues of t...,2018-01-25 16:17:44.789555,2018-02-02 01:19:41.756632,2018-02-02 01:19:41.756664,"“Oh, Trump, you coward, you just wait, we will...","F.N. Lehner, Don Spilman, Clarence J. Feinour,...",NaN,[''],NaN,NaN,NaN,NaN


In [8]:
columns_to_drop = [
    'Unnamed: 0', 'id', 'url', 'scraped_at', 'inserted_at', 'updated_at', 'meta_keywords', 'source', 'authors', 'keywords','meta_description', 'tags','summary'
]

In [9]:
df = df.drop(columns=columns_to_drop)


In [10]:
df.head()

,domain,type,content,title
0,express.co.uk,rumor,"Life is an illusion, at least on a quantum lev...",Is life an ILLUSION? Researchers prove 'realit...
1,barenakedislam.com,hate,"Unfortunately, he hasn’t yet attacked her for ...",Donald Trump
2,barenakedislam.com,hate,The Los Angeles Police Department has been den...,Donald Trump
3,barenakedislam.com,hate,The White House has decided to quietly withdra...,"MORE WINNING! Israeli intelligence source, DEB..."
4,barenakedislam.com,hate,“The time has come to cut off the tongues of t...,"“Oh, Trump, you coward, you just wait, we will..."


In [11]:
unique_targets = df['type'].unique()
unique_targets

array(['rumor', 'hate', 'unreliable', 'conspiracy', 'clickbait', 'satire',
       'fake', 'reliable', 'bias', 'political', 'junksci', nan, 'unknown',
       '2018-02-02 01:19:41.756664', '2018-02-07 23:39:33.852696',
       '2017-11-10T11:18:44.524042', '2018-02-08 19:18:34.468066',
       ' including Iraq', '2018-02-08 19:18:34.468038',
       '” said Linton Roberts', '2018-02-10 13:43:39.521661'],
      dtype=object)

In [12]:
relevant_labels = ['fake', 'reliable','bias','conspiracy','clickbait','unreliable']

df_filtered = df[df['type'].isin(relevant_labels)]

print("Updated unique target variables:")
print(df_filtered['type'].unique())

print("\nFrequency of each target variable:")
print(df_filtered['type'].value_counts())


Updated unique target variables:
['unreliable' 'conspiracy' 'clickbait' 'fake' 'reliable' 'bias']

Frequency of each target variable:
type
reliable      1913222
bias          1138998
fake           894746
conspiracy     831235
unreliable     298784
clickbait      231949
Name: count, dtype: int64


In [13]:
nan_rows = df[df.isna().any(axis=1)]

print("Rows with NaN values:")
print(nan_rows)

print("\nNumber of NaN rows:", len(nan_rows))

Rows with NaN values:
                                 domain                        type  \
397                     www.cnsnews.com                         NaN   
663                      freebeacon.com                         NaN   
664                      freebeacon.com                         NaN   
3097               www.greenmedinfo.com                         NaN   
3098               www.greenmedinfo.com                         NaN   
...                                 ...                         ...   
9977557   2016-11-14T21:16:20.914+02:00  2018-02-10 13:43:39.521661   
9985384                     www.cnn.com                    reliable   
9991838                     www.cnn.com                    reliable   
10002694                www.reuters.com                    reliable   
10002695  2016-11-17T20:50:20.601+02:00  2018-02-10 13:43:39.521661   

                                                    content  \
397       Sen. Dick Durbin (D-Ill.) (Screenshot of C-SPA...   

In [14]:
# Drop rows with NaN values in any column
df_filtered = df.dropna()

print("Number of NaN rows after dropping:", df_filtered.isna().sum().sum())

Number of NaN rows after dropping: 0


In [15]:
df_filtered.head()

,domain,type,content,title
0,express.co.uk,rumor,"Life is an illusion, at least on a quantum lev...",Is life an ILLUSION? Researchers prove 'realit...
1,barenakedislam.com,hate,"Unfortunately, he hasn’t yet attacked her for ...",Donald Trump
2,barenakedislam.com,hate,The Los Angeles Police Department has been den...,Donald Trump
3,barenakedislam.com,hate,The White House has decided to quietly withdra...,"MORE WINNING! Israeli intelligence source, DEB..."
4,barenakedislam.com,hate,“The time has come to cut off the tongues of t...,"“Oh, Trump, you coward, you just wait, we will..."


In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import pickle

valid_types = ['reliable', 'bias', 'fake', 'conspiracy', 'unreliable', 'clickbait']
df_filtered = df[df['type'].isin(valid_types)].copy()
df_balanced = df_filtered.groupby('type', group_keys=False).apply(
    lambda x: x.sample(n=min(50000, len(x)), random_state=42)
).reset_index(drop=True)

lr_label_encoder = LabelEncoder()
y = lr_label_encoder.fit_transform(df_balanced['type'])

texts = (
    df_balanced['domain'].fillna('') + " " +
    df_balanced['title'].fillna('') + " " +
    df_balanced['content'].fillna('')
)

# TF-IDF vectorization
tfidf = TfidfVectorizer(max_features=50000)
X = tfidf.fit_transform(texts)

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


<ipython-input-17-8746ab275aac>:10: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df_filtered.groupby('type', group_keys=False).apply(


In [18]:
lr_model = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=1000)
lr_model.fit(X_train, y_train)

# Predict probabilities
y_pred_probs_lr = lr_model.predict_proba(X_test)
y_pred_lr = lr_model.predict(X_test)

# Evaluate
print(classification_report(
    y_test, y_pred_lr,
    labels=range(len(lr_label_encoder.classes_)),
    target_names=lr_label_encoder.classes_,
    zero_division=0
))

# Save model and TF-IDF vectorizer
with open("lr_model.pkl", "wb") as f:
    pickle.dump(lr_model, f)

with open("tfidf_vectorizer.pkl", "wb") as f:
    pickle.dump(tfidf, f)

with open("lr_label_encoder.pkl", "wb") as f:
    pickle.dump(lr_label_encoder, f)


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


              precision    recall  f1-score   support

        bias       0.91      0.94      0.93     10009
   clickbait       0.94      0.97      0.96      9918
  conspiracy       0.94      0.94      0.94      9980
        fake       0.96      0.94      0.95     10035
    reliable       0.97      0.97      0.97     10136
  unreliable       0.99      0.94      0.96      9922

    accuracy                           0.95     60000
   macro avg       0.95      0.95      0.95     60000
weighted avg       0.95      0.95      0.95     60000



In [21]:
!pip install tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.9/644.9 MB 612.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 104.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 116.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00


In [22]:
import pickle
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import load_model

# Load CNN
cnn_model = load_model("/content/cnn_balanced_model.h5")
with open("/content/CNN_tokenizer.pkl", "rb") as f:
    cnn_tokenizer = pickle.load(f)
with open("/content/CNN_label_encoder.pkl", "rb") as f:
    cnn_label_encoder = pickle.load(f)

# Load LSTM
lstm_model = load_model("/content/lstm_balanced_model.h5")
with open("/content/lstm_tokenizer.pkl", "rb") as f:
    lstm_tokenizer = pickle.load(f)
with open("/content/lstm_label_encoder.pkl", "rb") as f:
    lstm_label_encoder = pickle.load(f)

# Load Logistic Regression
with open("/content/lr_model.pkl", "rb") as f:
    lr_model = pickle.load(f)
with open("/content/tfidf_vectorizer.pkl", "rb") as f:
    tfidf_vectorizer = pickle.load(f)
with open("/content/lr_label_encoder.pkl", "rb") as f:
    lr_label_encoder = pickle.load(f)

In [23]:
assert np.array_equal(
    cnn_label_encoder.classes_,
    lstm_label_encoder.classes_
) and np.array_equal(
    cnn_label_encoder.classes_,
    lr_label_encoder.classes_
), "Label classes must match in all models!"


In [24]:
def ensemble_predict(text, cnn_model, cnn_tokenizer, lstm_model, lstm_tokenizer, lr_model, tfidf_vectorizer, label_encoder, max_length=300):

    # CNN preprocessing
    cnn_seq = cnn_tokenizer.texts_to_sequences([text])
    cnn_input = pad_sequences(cnn_seq, maxlen=max_length)
    cnn_probs = cnn_model.predict(cnn_input, verbose=0)[0]

    # LSTM preprocessing
    lstm_seq = lstm_tokenizer.texts_to_sequences([text])
    lstm_input = pad_sequences(lstm_seq, maxlen=max_length)
    lstm_probs = lstm_model.predict(lstm_input, verbose=0)[0]

    # LR preprocessing
    lr_input = tfidf_vectorizer.transform([text])
    lr_probs = lr_model.predict_proba(lr_input)[0]

    # Average probabilities
    avg_probs = (cnn_probs + lstm_probs + lr_probs) / 3
    pred_class = np.argmax(avg_probs)

    return label_encoder.classes_[pred_class], avg_probs

In [25]:
text = "cnn.com Breaking: New vaccine study shows surprising results."
label, probs = ensemble_predict(
     text,
    cnn_model, cnn_tokenizer,
    lstm_model, lstm_tokenizer,
    lr_model, tfidf_vectorizer,
    cnn_label_encoder
)

print("Predicted class:", label)
print("Class probabilities:", dict(zip(cnn_label_encoder.classes_, probs.round(3))))

Predicted class: conspiracy
Class probabilities: {'bias': np.float64(0.028), 'clickbait': np.float64(0.025), 'conspiracy': np.float64(0.848), 'fake': np.float64(0.02), 'reliable': np.float64(0.018), 'unreliable': np.float64(0.062)}


In [26]:
text = "breitbart.com Breaking: Experts claim climate change is exaggerated by scientists."

label, probs = ensemble_predict(
    text,
    cnn_model, cnn_tokenizer,
    lstm_model, lstm_tokenizer,
    lr_model, tfidf_vectorizer,
    cnn_label_encoder
)

print("Predicted label:", label)
print("Class probabilities:", dict(zip(cnn_label_encoder.classes_, probs.round(3))))


Predicted label: conspiracy
Class probabilities: {'bias': np.float64(0.108), 'clickbait': np.float64(0.039), 'conspiracy': np.float64(0.796), 'fake': np.float64(0.027), 'reliable': np.float64(0.016), 'unreliable': np.float64(0.015)}


In [30]:
# Ensemble Prediction on Validation Set


valid_types = ['reliable', 'bias', 'fake', 'conspiracy', 'unreliable', 'clickbait']
df_filtered = df[df['type'].isin(valid_types)].copy()
df_balanced = df_filtered.groupby('type', group_keys=False).apply(
    lambda x: x.sample(n=min(50000, len(x)), random_state=42)
).reset_index(drop=True)

# (C) Encode labels
from sklearn.preprocessing import LabelEncoder
true_encoder = LabelEncoder()
y_true = true_encoder.fit_transform(df_balanced['type'])

# Confirm it matches
assert list(true_encoder.classes_) == list(cnn_label_encoder.classes_), "Label mismatch!"

# (D) Prepare text
texts = (
    df_balanced['domain'].fillna('') + ' ' +
    df_balanced['title'].fillna('') + ' ' +
    df_balanced['content'].fillna('')
)




<ipython-input-30-b4dcf30fbeb4>:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_balanced = df_filtered.groupby('type', group_keys=False).apply(


In [31]:
# Step 1: Tokenize all texts once for each model

cnn_sequences = cnn_tokenizer.texts_to_sequences(texts)
cnn_padded = pad_sequences(cnn_sequences, maxlen=300)

lstm_sequences = lstm_tokenizer.texts_to_sequences(texts)
lstm_padded = pad_sequences(lstm_sequences, maxlen=300)

lr_input = tfidf_vectorizer.transform(texts)


In [32]:
cnn_probs_all = cnn_model.predict(cnn_padded, verbose=1)
lstm_probs_all = lstm_model.predict(lstm_padded, verbose=1)
lr_probs_all = lr_model.predict_proba(lr_input)


9375/9375 ━━━━━━━━━━━━━━━━━━━━ 187s 20ms/step
9375/9375 ━━━━━━━━━━━━━━━━━━━━ 286s 30ms/step


In [34]:
avg_probs_all = (cnn_probs_all + lstm_probs_all + lr_probs_all) / 3
y_pred_ensemble = np.argmax(avg_probs_all, axis=1)


In [35]:
from sklearn.metrics import accuracy_score, f1_score

y_true_numeric = true_encoder.transform(df_balanced['type'])

print(f"Ensemble Accuracy: {accuracy_score(y_true_numeric, y_pred_ensemble):.4f}")
print(f"Ensemble Macro F1: {f1_score(y_true_numeric, y_pred_ensemble, average='macro'):.4f}")


Ensemble Accuracy: 0.9850
Ensemble Macro F1: 0.9850


In [36]:
# CNN
cnn_preds = np.argmax(cnn_probs_all, axis=1)
print("CNN Accuracy:", accuracy_score(y_true_numeric, cnn_preds))
print("CNN Macro F1:", f1_score(y_true_numeric, cnn_preds, average='macro'))

# LSTM
lstm_preds = np.argmax(lstm_probs_all, axis=1)
print("LSTM Accuracy:", accuracy_score(y_true_numeric, lstm_preds))
print("LSTM Macro F1:", f1_score(y_true_numeric, lstm_preds, average='macro'))

# LR
lr_preds = np.argmax(lr_probs_all, axis=1)
print("LR Accuracy:", accuracy_score(y_true_numeric, lr_preds))
print("LR Macro F1:", f1_score(y_true_numeric, lr_preds, average='macro'))


CNN Accuracy: 0.98334
CNN Macro F1: 0.9833487708662711
LSTM Accuracy: 0.8879333333333334
LSTM Macro F1: 0.888282578710581
LR Accuracy: 0.9672033333333333
LR Macro F1: 0.9672610548864169
